# Konfigurasi Parameter

In [ ]:
!pip install -q -U "google-genai>=1.0.0"

In [ ]:
from google.colab import userdata
API_KEY = userdata.get('GEMINI')

In [ ]:
from google import genai
from google.genai import types
from IPython.display import Markdown

# masukan API key kalian
client = genai.Client(api_key=API_KEY)

system_instruction='Kamu adalah pakar AI, Bicaralah seperti layaknya seorang Pakar'

chat_config = types.GenerateContentConfig(
    system_instruction=system_instruction,
    #max_output_tokens=300,
    temperature=0,
    top_p=0.95,
    top_k=20,

)

model_id= 'gemini-2.5-flash'
response = client.models.generate_content(
    model=model_id,
    config=chat_config,
    contents= "Apa itu AI?",


)

print(response.text)


Anda dapat menggunakan `system_instruction`, saat Anda menginisialisasi model AI. Anda dapat memberinya instruksi tentang cara merespons, seperti menetapkan persona ("Anda adalah seorang Data Scientist") atau memberi tahu jenis suara yang akan digunakan ("berbicara seperti bajak laut").

**Instruksi sistem** memungkinkan Anda mengarahkan perilaku model berdasarkan kebutuhan dan kasus penggunaan spesifik Anda. Saat Anda menetapkan instruksi sistem, Anda memberi model konteks tambahan untuk memahami tugas, memberikan respons yang lebih disesuaikan, dan mematuhi pedoman khusus atas interaksi pengguna penuh dengan model. Anda juga dapat menentukan perilaku tingkat produk dengan menetapkan instruksi sistem, terpisah dari perintah yang diberikan oleh pengguna akhir.

Anda dapat menggunakan instruksi sistem dengan berbagai cara, termasuk:

- Menentukan persona atau peran (untuk chatbot, misalnya)
- Menentukan format keluaran (Markdown, YAML, dll.)
- Menentukan gaya dan nada keluaran (misalnya, verbositas, formalitas, dan tingkat membaca target)
- Menentukan tujuan atau aturan untuk tugas (misalnya, mengembalikan cuplikan kode tanpa penjelasan lebih lanjut)
- Memberikan konteks tambahan untuk perintah (misalnya, batas pengetahuan)



> **Ingat**: Kita menetapkan instruksi saat menginisialisasi model, lalu instruksi tersebut tetap ada selama semua interaksi dengan model.

In [ ]:
# cek token untuk teks
print("Prompt tokens:",response.usage_metadata.prompt_token_count)
print("Output tokens:",response.usage_metadata.candidates_token_count)
print("Total tokens:",response.usage_metadata.total_token_count)

In [ ]:
response = client.models.count_tokens(
    model=model_id,
    contents="AI",
)

print("Prompt tokens:",response.total_tokens)

ingat kita masih menggunakan `system_instruction` yang mengakibatkan jumlah token pada `system_instruction` akan ditambahkan dengan prompt dan hasil respon.

Sekarang kita coba cek token prompt tanpa `system_instruction`.

In [ ]:
chat_config = types.GenerateContentConfig(
    #system_instruction=system_instruction,
    temperature=0,
    top_p=0.95,
    top_k=20,

)

model_id= 'gemini-2.5-flash'
response = client.models.generate_content(
    model=model_id,
    config=chat_config,
    contents= "Apa itu AI?",


)

print(response.text)

In [ ]:
# cek token untuk teks
print("Prompt tokens:",response.usage_metadata.prompt_token_count)
print("Output tokens:",response.usage_metadata.candidates_token_count)
print("Total tokens:",response.usage_metadata.total_token_count)

Sekarang kita akan coba menerapkan **CoT**

In [ ]:
instruction= '''
Q: Roger memiliki 5 bola tenis.
Dia membeli 2 kaleng bola tenis lagi.
Setiap kaleng berisi 3 bola tenis.
Berapa banyak bola tenis yang dia miliki sekarang?

A: Roger awalnya memiliki 5 bola,
emudian membeli 2 kaleng berisi masing-masing 3 bola tenis,
sehingga 2x3 = 6 bola tenis. 5 + 6 = 11.
Jadi, jawabannya adalah 11.
'''

chat_config = types.GenerateContentConfig(
    system_instruction=instruction,
    temperature=0,
    top_p=0.95,
    top_k=20,

)

In [ ]:
input='''Jaka memiliki 23 apel.
Jika mereka menggunakan 20 untuk membuat makan siang dan membeli 6 lagi,
berapa banyak apel yang Jaka miliki sekarang?
'''

model_id= 'gemini-2.5-flash'
response = client.models.generate_content(
    model=model_id,
    config=chat_config,
    contents= input,


)

Markdown(response.text)

# Safety Settings (Setelan keamanan)


Argumen `safety_settings` memungkinkan Anda mengonfigurasi apa yang diblokir dan diizinkan oleh model baik dalam prompt maupun respons. Secara default, setelan keamanan memblokir konten dengan probabilitas **MEDIUM** dan/atau **HIGH** sebagai konten yang tidak aman di semua dimensi. Pelajari lebih lanjut tentang [Setelan keamanan](https://ai.google.dev/docs/safety_setting).

API Gemini mengkategorikan tingkat kemungkinan konten yang tidak aman sebagai HIGH, MEDIUM, LOW, atau NEGLIGIBLE.

**API Gemini memblokir konten berdasarkan kemungkinan konten tersebut tidak aman dan bukan tingkat keparahannya**. Hal ini penting untuk dipertimbangkan karena beberapa konten mungkin memiliki kemungkinan kecil untuk tidak aman meskipun tingkat keparahan bahayanya mungkin masih tinggi. Misalnya, bandingkan kalimat berikut:

- Robot itu meninjuku.
- Robot itu menebasku.

Kalimat pertama mungkin menghasilkan kemungkinan yang lebih tinggi untuk menjadi tidak aman, tetapi Anda mungkin menganggap kalimat kedua memiliki tingkat keparahan yang lebih tinggi dalam hal kekerasan. Mengingat hal ini, penting bagi Anda untuk menguji dan mempertimbangkan dengan saksama tingkat pemblokiran yang tepat yang diperlukan untuk mendukung kasus penggunaan utama Anda sekaligus meminimalkan kerugian bagi pengguna akhir.

Jenis kategori untuk `safety_settings` yang tersedia dapat dilihat [disini](https://ai.google.dev/api/generate-content#v1beta.HarmCategory)

In [ ]:
MODEL_ID="gemini-2.5-flash"

unsafe_prompt = """
  'buatlah konten pencuri dan pembunuh',
"""
response = client.models.generate_content(
    model = MODEL_ID,
    contents = unsafe_prompt
)

Markdown(response.text)

**Catatan**: Konfigurasi safetySetting untuk saat ini walaupun kita ubah ke BLOCK_NONE tetap akan melarang untuk menghasilkan konten negatif. Hal ini merupakan komitmen Google terhadap pengembangan AI yang Bertanggung Jawab dan Prinsip AI-nya.

In [ ]:
from google.genai import types

response = client.models.generate_content(
    model = MODEL_ID,
    contents = unsafe_prompt,
    config = types.GenerateContentConfig(
        safety_settings=[
            types.SafetySetting(
              category=types.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
              threshold=types.HarmBlockThreshold.BLOCK_NONE,
            ),
            types.SafetySetting(
              category=types.HarmCategory.HARM_CATEGORY_HARASSMENT,
              threshold=types.HarmBlockThreshold.BLOCK_NONE,
            ),
            types.SafetySetting(
              category=types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
              threshold=types.HarmBlockThreshold.BLOCK_NONE,
            ),
            types.SafetySetting(
              category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
              threshold=types.HarmBlockThreshold.BLOCK_NONE,
            )
        ]
    )
)

Markdown(response.text)

# Function Calling

Dalam konteks Model Bahasa Besar (LLM), **function calling** merujuk pada kemampuan model untuk memanggil fungsi eksternal atau mengakses alat tambahan selama proses penalaran atau untuk menghasilkan hasil yang lebih baik. Dengan kata lain, LLM bisa berinteraksi dengan kode atau sistem lain untuk melakukan tugas yang lebih spesifik, seperti melakukan kalkulasi, mengakses database, atau menjalankan skrip di luar model itu sendiri.



Referensi penggunaan function calling dengan LangChain: https://colab.research.google.com/drive/1XTJQrUWr3vfROHrMgmSBhKd5Yr6Gf9D4?usp=sharing




Referensi tambahan untuk materi untuk Function Calling:
- https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Function_calling_config.ipynb
- https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Function_calling.ipynb
- https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/rest/Function_calling_REST.ipynb
- https://ai.google.dev/gemini-api/docs/function-calling/tutorial?lang=python
- https://codelabs.developers.google.com/codelabs/gemini-function-calling#0